<div style="padding: -5px;
  text-align: center;
  color: white;
  font-size: 15px;">
   <img src="images/banner.jpg" alt="MINE-Seminario de programación" style="width:100%;">
  <h1 style="
  position: absolute;
  top: 5%;
  left: 50%;">Paquete Prince</h1>
</div>

## Otro paquete: Prince

Veremos algunas aplicaciones elementales del paquete prince. 

Prince es una biblioteca para hacer análisis factorial. Esto incluye una variedad de métodos, incluido el análisis de componentes principales (PCA) y el análisis de correspondencia (CA). El objetivo es proporcionar una implementación eficiente para cada algoritmo junto con una API scikit-learn.


## Consideraciones

Cada estimador proporcionado por prince amplía TransformerMixin de scikit-learn. Esto significa que cada estimador implementa un a función de ajuste y transformación que los hace utilizables en un proceso de transformación. La función de ajuste es en realidad un alias del método row_principal_components que devuelve los componentes principales de la fila. Sin embargo, también puede acceder a los componentes principales de la columna con column_principal_components.

La programación de Prince usa una versión aleatoria de SVD. Esto es mucho más rápido que utilizar el enfoque completo más común. Sin embargo, los resultados pueden tener una pequeña aleatoriedad inherente. Para la mayoría de las aplicaciones, esto no importa y no debería tener que preocuparse por ello. Sin embargo, si desea resultados reproducibles, debe configurar el parámetro random_state.

La versión aleatoria de SVD es un método iterativo. Debido a que cada uno de los algoritmos de Prince usa SVD, todos poseen un parámetro n_iter que controla el número de iteraciones utilizadas para calcular el SVD. Por un lado, cuanto mayor sea n_iter, más precisos serán los resultados. Por otro lado, aumentar n_iter aumenta el tiempo de cálculo. En general, el algoritmo converge muy rápidamente, por lo que se recomienda utilizar un n_iter bajo (que es el comportamiento predeterminado).

Se supone que debe usar cada método según su situación:

* Todas sus variables son numéricas: utilice el análisis de componentes principales (prince.PCA)
* Tienes una tabla de contingencia: usa el análisis de correspondencia (prince.CA)
*  Tiene más de 2 variables y todas son categóricas: use análisis de correspondencia múltiple (prince.MCA)
* Tiene grupos de variables categóricas o numéricas: use análisis de factores múltiples (prince.MFA)
* Tiene variables categóricas y numéricas: utilice el análisis factorial de datos mixtos (prince.FAMD)


## Análisis de componentes principales

Todas las variables numéricas

In [0]:
pip install prince

In [0]:
import pandas as pd
import prince
from sklearn import datasets

X, y = datasets.load_iris(return_X_y=True)
X = pd.DataFrame(data=X, columns=['Sepal length', 'Sepal width', 'Petal length', 'Petal width'])
y = pd.Series(y).map({0: 'Setosa', 1: 'Versicolor', 2: 'Virginica'})

In [0]:
X

In [0]:
pca = prince.PCA(
    n_components=2,
     n_iter=3,
     rescale_with_mean=True,
     rescale_with_std=True,
     copy=True,
     check_input=True,
     engine='auto',
     random_state=42
 )
pca = pca.fit(X)


Los parámetros disponibles son:

* n_components: el número de componentes que se calculan. Solo necesita dos si su intención es hacer un gráfico.
* n_iter: el número de iteraciones utilizadas para calcular la SVD
* rescale_with_mean: si restar la media de cada columna
* rescale_with_std: si dividir cada columna por su desviación estándar
* copy: si es False, los cálculos se realizarán en el lugar, lo que puede tener posibles efectos secundarios en los datos de entrada
* engine: qué motor SVD usar (debe ser uno de ['auto', 'fbpca', 'sklearn'])
* random_state: controla la aleatoriedad de los resultados de SVD.

Una vez que se ha ajustado el PCA, se puede utilizar para extraer las coordenadas principales de la fila de la siguiente manera:

In [0]:
pca.transform(X)

Cada columna representa un componente principal, mientras que cada fila representa una fila en el conjunto de datos original. Puede mostrar estas proyecciones con el comando plot_row_coordinates:

In [0]:
y

In [0]:
ax = pca.plot_row_coordinates(
   X,
     ax=None,
     figsize=(6, 6),
     x_component=0,
     y_component=1,
     color_labels=y,
     ellipse_outline=False,
     ellipse_fill=True,
     show_points=True
 )

Cada componente principal explica parte del subyacente de la distribución. Puede ver cuánto utilizando el acceso a la explained_inertia_ property:

In [0]:
pca.explained_inertia_ 

La inercia explicada representa el porcentaje de la inercia que aporta cada componente principal. Suma hasta 1 si la propiedad n_components es igual al número de columnas del conjunto de datos original. La inercia explicada se obtiene dividiendo los autovalores obtenidos con la SVD por la inercia total, ambos también accesibles.

In [0]:
pca.eigenvalues_

In [0]:
pca.total_inertia_  

In [0]:
pca.explained_inertia_

También puede obtener las correlaciones entre las variables originales y los componentes principales.

In [0]:
pca.column_correlations(X)

También es posible saber cuánto contribuye cada observación a cada componente principal. Esto se puede hacer con el comando row_contributions.

In [0]:
pca.row_contributions(X)

También puede transformar las proyecciones de filas a su espacio original utilizando la función inverse_transform.

In [0]:
pca.inverse_transform(pca.transform(X))

## Análisis de correspondencias

El análisis de correspondencia se utiliza cuando se desea analizar una tabla de contingencia. En otras palabras, buscamos analizar las dependencias entre dos variables categóricas.

In [0]:
import pandas as pd

pd.set_option('display.float_format', lambda x: '{:.6f}'.format(x))
X = pd.DataFrame(
   data=[
       [426, 345, 0, 11, 3],
       [54, 110, 45, 0, 4],
       [31, 84, 409, 41, 26],
       [12, 28, 40, 381, 5]],
   columns=pd.Series(['Fair', 'Red', 'Medium', 'Dark', 'Black']),
   index=pd.Series(['Blue', 'Light', 'Medium', 'Dark']))
X


In [0]:
X1=X[["Fair","Red","Dark","Black"]]
X1=X1.iloc[[0,1,3]]
X1

In [0]:
import matplotlib.pyplot as plt 
import seaborn as sns
fig, ax = plt.subplots()
s=sns.heatmap(X,cmap='coolwarm') 
ax.set_xlim(0,5)
ax.set_ylim(0,4)
plt.show()



In [0]:
ca = prince.CA(
    n_components=2,
    n_iter=5,
    copy=True,
    check_input=True,
    engine='auto',
    random_state=42)

In [0]:
X.columns.rename('Hair color', inplace=True)
X.index.rename('Eye color', inplace=True)
X

In [0]:
ca = ca.fit(X)

In [0]:
ca.row_coordinates(X)

In [0]:
ca.column_coordinates(X)

Puede trazar ambos conjuntos de coordenadas principales con el método plot_coordinates.

In [0]:
ax = ca.plot_coordinates(
    X=X,
    ax=None,
    figsize=(6, 6),
    x_component=0,
    y_component=1,
    show_row_labels=True,
    show_col_labels=True)

In [0]:
ca.eigenvalues_ 

In [0]:
ca.total_inertia_

In [0]:
ca.explained_inertia_

## Análisis de correspondencias multiples

El análisis de correspondencia múltiple (ACM) es una extensión del análisis de correspondencia (AC). Debería utilizarse cuando tenga más de dos variables categóricas. La idea es simplemente calcular los one-hot-vector de un conjunto de datos y aplicar CA en él. Como ejemplo, usaremos el conjunto de datos de [globos](https://archive.ics.uci.edu/ml/machine-learning-databases/balloons/) tomados del sitio web de conjuntos de datos de UCI.

In [0]:
X = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/balloons/adult+stretch.data')
X.columns = ['Color', 'Size', 'Action', 'Age', 'Inflated']
pd.get_dummies(X)

In [0]:
mca = prince.MCA(
    n_components=4,
    n_iter=3,
    copy=True,
    check_input=True,
    engine='auto',
    random_state=42)
mca = mca.fit(X)


Usamos nuevamente plot_coordinates

In [0]:
ax = mca.plot_coordinates(
    X=X,
    ax=None,
    x_component=1,
    y_component=3,
    figsize=(10, 10),
    show_row_points=False,
    show_row_labels=False,
    show_column_points=True,
    column_points_size=50,
    show_column_labels=True,
    legend_n_cols=2)

In [0]:
mca.column_coordinates(X)

### Análisis de factores múltiple

El análisis de factores múltiples (MFA) está diseñado para usarse cuando tiene grupos de variables. En la práctica, construye un PCA en cada grupo, o un MCA, según los tipos de variables del grupo. Luego, construye un PCA global sobre los resultados de los llamados PCA parciales, o MCA.

El conjunto de datos utilizado en los siguientes ejemplos proviene de este [documento](https://www.utdallas.edu/%7Eherve/Abdi-MFA2007-pretty.pdf). En el conjunto de datos, tres expertos dan su opinión sobre seis vinos diferentes. Cada opinión para cada vino se registra como una variable. Por lo tanto, queremos considerar las opiniones separadas de cada experto y al mismo tiempo tener una visión global de cada vino. MFA es el método perfecto para este tipo de situación.

Primero que nada, copiemos los datos usados en el documento.

In [0]:
X = pd.DataFrame(
    data=[
        [1, 6, 7, 2, 5, 7, 6, 3, 6, 7],
        [5, 3, 2, 4, 4, 4, 2, 4, 4, 3],
        [6, 1, 1, 5, 2, 1, 1, 7, 1, 1],
        [7, 1, 2, 7, 2, 1, 2, 2, 2, 2],
        [2, 5, 4, 3, 5, 6, 5, 2, 6, 6],
        [3, 4, 4, 3, 5, 4, 5, 1, 7, 5]],
    columns=['E1 fruity', 'E1 woody', 'E1 coffee',
             'E2 red fruit', 'E2 roasted', 'E2 vanillin', 'E2 woody',
             'E3 fruity', 'E3 butter', 'E3 woody'],
    index=['Wine {}'.format(i+1) for i in range(6)])
X['Oak type'] = [1, 2, 2, 2, 1, 1]
X

Los grupos se pasan como un diccionario a la clase MFA.

In [0]:
groups = {
   'Expert #{}'.format(no+1): [c for c in X.columns if c.startswith('E{}'.format(no+1))]
   for no in range(3)
}
print(groups)

Ajustamos un MFA

In [0]:
mfa = prince.MFA(
    groups=groups,
    n_components=2,
    n_iter=3,
    copy=True,
    check_input=True,
    random_state=42)
mfa = mfa.fit(X)


El MFA hereda de la clase PCA, lo que implica que tiene acceso a todos sus métodos y propiedades. El método row_coordinates devolverá las coordenadas globales de cada vino.

In [0]:
mfa.row_coordinates(X)

Al igual que para el PCA, puede trazar las coordenadas de la fila con el comando plot_row_coordinates.

In [0]:
ax = mfa.plot_row_coordinates(
    X,
    ax=None,
    figsize=(6, 6),
    x_component=0,
    y_component=1,
    labels=X.index,
    color_labels=['Oak type {}'.format(t) for t in X['Oak type']],
    ellipse_outline=False,
    ellipse_fill=True,
    show_points=True)

También puede obtener las coordenadas de las filas dentro de cada grupo. El método parcial_row_coordinates devuelve un pandas.DataFrame donde el conjunto de columnas es un pandas.MultiIndex. El primer nivel de indexación corresponde a cada grupo especificado, mientras que el nivel anidado indica las coordenadas dentro de cada grupo.

In [0]:
mfa.partial_row_coordinates(X)

plot_partial_row_coordinates permite visualizar la opinion de cada experto

In [0]:
?mfa.plot_partial_row_coordinates

In [0]:
ax = mfa.plot_partial_row_coordinates(
    X,
    ax=None,
    figsize=(6, 6),
    x_component=0,
    y_component=1,
    color_labels=['Oak type {}'.format(t) for t in X['Oak type']])

In [0]:
mfa.partial_row_coordinates(X)

In [0]:
mfa.eigenvalues_

In [0]:
mfa.total_inertia_

In [0]:
mfa.explained_inertia_

In [0]:
for name, fa in sorted(mfa.partial_factor_analysis_.items()): 
    print('{} inertia: {}'.format(name, fa.explained_inertia_))